In [1]:
from crewai_tools import ScrapeWebsiteTool, FileWriterTool, TXTSearchTool
import requests
from crewai import Agent, Task, Crew, Process, LLM
import os 
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_

In [2]:
# Initialize the tool, potentially passing the session
scrape_tool = ScrapeWebsiteTool(website_url='https://en.wikipedia.org/wiki/Artificial_intelligence')

# Extract the text
text = scrape_tool.run()
print(text[:100])

Using Tool: Read website content

Artificial intelligence - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
Naviga


In [3]:
# Initialize the tool
# file_writer_tool = FileWriterTool()

# Write content to a file in a specified directory
# result = file_writer_tool.run(filename='ai.txt', content = text, directory = '', overwrite=True)
# print(result)

# Set file path
base_path = os.path.join('.', 'ai.txt')

# Save file content
with open(base_path, 'w', encoding='utf-8') as f:
    f.write(text)
print(f"Content saved to: {base_path}")

Content saved to: ./ai.txt


In [ ]:
# Load a local embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Choose a model you like

# Define a function to generate embeddings
def embed_fn(texts):
    return embedding_model.encode(texts).tolist()

ollama = LLM(
 model="ollama/deepseek-r1:8b",
 base_url="http://host.docker.internal:11434"
)

# Initialize the tool with a specific text file, so the agent can search within the given text file's content
# uses chromadb to chunk and vectorize data
search_tool = TXTSearchTool(txt='ai.txt', embed_fn=embed_fn, chunk_size=256)

Inserting batches in chromadb:   0%|          | 0/2 [00:00<?, ?it/s]


In [8]:
data_analyst = Agent(
    role='Educator',
    goal=f'Based on the context provided, answer the question.',
    backstory='You are a data expert',
    verbose=True,
    allow_delegation=False,
    llm=ollama,
    tools=[search_tool]
)

test_task = Task(
    description="Understand the topic of Natural Language Processing and summarize it for me",
    agent=data_analyst,
    expected_output='I want the response to be as short as possible.'
)

crew = Crew(
    tasks=[test_task],
    process=Process.sequential,
    model = "ollama/deepseek-r1:8b",
)

output = crew.kickoff()

2025-02-26 10:13:00,856 - 140093469862784 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Educator
## Task: Understand the topic of Natural Language Processing and summarize it for me


# Agent: Educator
## Final Answer: 
**
Natural Language Processing (NLP) is a branch of computer science focused on enabling computers to understand, interpret, and generate human language. It involves processes such as tokenization, parsing, Bag of Words, and the use of machine learning algorithms to analyze text data, perform tasks like sentiment analysis, and generate responses. NLP applications include chatbots, information extraction, and text generation, utilizing tools like TensorFlow, PyTorch, NLTK, and SpaCy.


